# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kalemie,-5.9475,29.1947,77.97,46,23,13.78,CD,1690094841
1,1,callao,-12.0667,-77.1500,70.09,73,100,5.75,PE,1690094842
2,2,bethel,41.3712,-73.4140,67.50,85,0,3.44,US,1690094843
3,3,qaqortoq,60.7167,-46.0333,53.19,73,100,4.94,GL,1690094843
4,4,grytviken,-54.2811,-36.5092,24.08,79,59,11.01,GS,1690094845
...,...,...,...,...,...,...,...,...,...,...
567,567,zouerate,22.7187,-12.4521,74.97,49,5,9.22,MR,1690095197
568,568,victorica,-36.2151,-65.4359,48.47,49,100,16.06,AR,1690095198
569,569,khorixas,-20.3667,14.9667,68.07,33,0,6.51,NaN,1690095198
570,570,tadmur,34.5608,38.2805,96.40,21,0,14.43,SY,1690095199


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

#### Narrow down the city_data_df DataFrame to find your ideal weather condition. For example:
* A max temperature lower than 27 degrees but higher than 21
* Wind speed less than 4.5 m/s
* Zero cloudiness

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 70 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 70) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 10)]

# Drop results with null values
# city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,bethel,41.3712,-73.4140,67.50,85,0,3.44,US,1690094843
35,35,albany,42.6001,-73.9662,61.45,88,6,1.99,US,1690094866
65,65,jamestown,42.0970,-79.2353,61.72,95,0,3.44,US,1690094586
76,76,hamilton,39.1834,-84.5333,67.93,86,0,0.00,US,1690094891
121,121,kodiak,57.7900,-152.4072,56.71,82,0,0.00,US,1690094919


### Step 3: Create a new DataFrame called `hotel_df`.

#### Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.loc[:, ('City', 'Country','Lat','Lng','Humidity')]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# # Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,bethel,US,41.3712,-73.4140,85,
35,albany,US,42.6001,-73.9662,88,
65,jamestown,US,42.0970,-79.2353,95,
76,hamilton,US,39.1834,-84.5333,86,
121,kodiak,US,57.7900,-152.4072,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {"categories":categories,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"] 
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
albany - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
hamilton - nearest hotel: North Vista Manor
kodiak - nearest hotel: Shelikof Lodge
arvika - nearest hotel: Jennys Hotell och restaurang
broome - nearest hotel: No hotel found
sauk rapids - nearest hotel: AmericInn by Wyndham Sartell
filipstad - nearest hotel: Hertig Karl
lancaster - nearest hotel: Mountain Trail Motel
el calafate - nearest hotel: Hostería Las Avutardas
xanxere - nearest hotel: City Hotel
shurugwi - nearest hotel: Grand Hotel
pacific grove - nearest hotel: Pacific Grove Inn
richfield - nearest hotel: Four Points by Sheraton Minneapolis Airport
opuwo - nearest hotel: Ohakane Lodge
santa isabel - nearest hotel: No hotel found
sault ste. marie - nearest hotel: Holiday Inn
conceicao do araguaia - nearest hotel: No hotel found
mizque - nearest hotel: Hotel Bolivia
laela - nearest hotel: No hotel found
itupiranga - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
2,bethel,US,41.3712,-73.4140,85,Hampton Inn Danbury
35,albany,US,42.6001,-73.9662,88,No hotel found
65,jamestown,US,42.0970,-79.2353,95,DoubleTree Jamestown
76,hamilton,US,39.1834,-84.5333,86,North Vista Manor
121,kodiak,US,57.7900,-152.4072,82,Shelikof Lodge
129,arvika,SE,59.6553,12.5852,77,Jennys Hotell och restaurang
136,broome,US,42.2506,-75.8330,86,No hotel found
160,sauk rapids,US,45.5919,-94.1661,93,AmericInn by Wyndham Sartell
174,filipstad,SE,59.7124,14.1683,66,Hertig Karl
189,lancaster,US,40.2504,-76.2499,89,Mountain Trail Motel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)